In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2
# Torch
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torchvision.transforms.functional as TF
# TB
from torch.utils.tensorboard import SummaryWriter
# CV2
import cv2
# TQDM
from tqdm import tqdm
# Summary
from torchsummary import summary

# Dataset

In [ ]:
# Dataset
class CTDataset(Dataset):
  def __init__(self, images, masks, transform=None):
      self.images = images
      self.masks = masks
      self.transform = transform
      #print(len(self.data))

  def __len__(self):
    return len(self.images)

  def __getitem__(self, index):
    image = self.images[index]
    mask = self.masks[index]

    if self.transform is not None:
      image = image.astype(np.float32)
      mask = mask.astype(np.float32)
      #print(image.shape, image.dtype, mask.shape, mask.dtype)
      augmentation = self.transform(image=image, mask=mask)
      image = augmentation['image']
      mask = augmentation['mask']
      return image.float(), mask.float().unsqueeze(0)

    return np.expand_dims(image.astype(np.float32), axis=0), np.expand_dims(mask.astype(np.float32), axis=0)

def test_dataset():
  # No augmentation
  no_aug = CTDataset(val_images[:10], val_masks[:10])
  image, mask = no_aug[0]
  print(image.shape, image.dtype, mask.shape, mask.dtype)

  # With Augmentation
  testing_transformer = A.Compose([
            #A.Resize(height=160, width=160),
            #A.RandomCrop(width=500, height=500),
            A.Rotate(limit=20, p=0.5, border_mode=cv2.BORDER_CONSTANT, value=-2000),
            A.HorizontalFlip(p=0.5),
            #A.VerticalFlip(p=0.1),
            #A.Blur(blur_limit=1, p=0.5),
            # A.Normalize(
            #     mean=[0.0, 0.0, 0.0],
            #     std=[1.0, 1.0, 1.0],
            #     max_pixel_value=255.0,
            # ),
            ToTensorV2(),
            ])
  aug = CTDataset(val_images[:10], val_masks[:10], testing_transformer)
  image, mask = aug[5]
  print(image.shape, image.dtype, mask.shape, mask.dtype)
  plt.imshow(image[0], cmap="gray", origin="lower")

if __name__ == "__main__":
  pass
  #test_dataset()

# Models

In [ ]:
# Onet model
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        return F.relu(self.bn2(self.conv2(out)))


class ONET(nn.Module):
    def mp(self, x, kernel=2, stride=2):
        return F.max_pool2d(x, kernel, stride)

    def __init__(
        self,
        in_channels=1,
        out_channels=1,
        features=[64, 32]
        ):
        super(ONET, self).__init__()
        self.top_en = nn.ModuleList()
        self.top_de = nn.ModuleList()
        self.btm_en = nn.ModuleList()
        self.btm_de = nn.ModuleList()

        # self.ups = nn.ModuleList()
        # self.downs = nn.ModuleList()
        # self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Right part of ONET
        for feature in features:
            self.top_en.append(DoubleConv(in_channels, feature))
            self.top_en.append(nn.ConvTranspose2d(feature, feature, kernel_size=2, stride=2))

            self.btm_en.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Left part of ONET
        for feature in reversed(features):
            self.top_de.append(nn.Conv2d(feature//2, feature, 3, 1, 1))
            self.top_de.append(DoubleConv(feature*2, feature))

            self.btm_de.append(nn.ConvTranspose2d(feature//2, feature, kernel_size=2, stride=2))
            self.btm_de.append(DoubleConv(feature*2, feature))

        # Bottleneck
        self.top_bottleneck = DoubleConv(features[-1], features[-1]//2)
        self.btm_bottleneck = DoubleConv(features[-1], features[-1]//2)

        #Output
        self.final_conv = nn.Conv2d(features[0]*2, out_channels, kernel_size=1)

    def forward(self, x):
        b, t = x, x
        top_skip_connections = []
        btm_skip_connections = []

        # Bottom part of the ONET

        for down in self.btm_en:
            b = down(b)
            btm_skip_connections.append(b)
            b = self.mp(b)

        b = self.btm_bottleneck(b)
        btm_skip_connections = btm_skip_connections[::-1]

        for idx in range(0, len(self.btm_de), 2):
            b = self.btm_de[idx](b)
            skip_connection = btm_skip_connections[idx//2]
            if b.shape != skip_connection.shape:
                b = TF.resize(b, size=skip_connection.shape[2:])
            concat_skip = torch.cat((skip_connection, b), dim=1)
            b = self.btm_de[idx+1](concat_skip)
        # Top part of the ONET
        save = False
        for idx, up in enumerate(self.top_en):
            t = up(t)
            # print(idx, t.shape)
            if idx%2 !=0:
              # print('saved:', idx)
              top_skip_connections.append(t)
            save = !save
        t = self.top_bottleneck(t)
        # decoder

        top_skip_connections = top_skip_connections[::-1]

        for idx in range(0, len(self.top_de), 2):
            t = self.top_de[idx](t)
            skip_connection = top_skip_connections[idx//2]
            # print('t', idx, t.shape)
            # print('sc:', idx//2, skip_connection.shape)

            if t.shape != skip_connection.shape:
                # print('resizing')
                t = TF.resize(t, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, t), dim=1)
            # print('concat: ', concat_skip.shape)
            t = self.mp(self.top_de[idx+1](concat_skip))

            # print('ft:', idx, t.shape)
        # print(t.shape, b.shape, 'suuup')
        x = torch.cat((t, b), dim=1)
        return self.final_conv(x)

def test_model_o():
    x = torch.randn((5, 1, 160, 160)).to('cpu')
    model = ONET(1,1, features=[32,16]).to('cpu')
    preds = model(x)
    print('input ', preds.shape)
    print('output: ', x.shape)
    summary(model, input_size=(1, 160, 160))
    assert preds.shape == x.shape

if __name__ == '__main__':
    pass
    #test_model_o()

In [ ]:
# Load and save the model
def save_checkpoint(state, filename="CTUnet.pth.tar"):
    print(f"[*] Saving checkpoint: {filename}")
    torch.save(state, filename)

def load_checkpoint(checkpoint, model):
    print(f"[*] Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])

In [ ]:
# Building data loaders
def get_loaders(
    train_images,
    train_masks,
    val_images,
    val_masks,
    train_batch_size, 
    val_batch_size,
    train_transform,
    val_transform,
    num_workers=4,
    pin_memory=True):
  
  # Training data loader
  train_ds = CTDataset(
        images=train_images,
        masks=train_masks, 
        transform=train_transform)
  train_loader = DataLoader(
        train_ds,
        batch_size=train_batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=True)
  
  # Validation data loader
  val_ds = CTDataset(
        images=val_images,
        masks=val_masks, 
        transform=val_transform)
  val_loader = DataLoader(
        val_ds,
        batch_size=val_batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=False)
    
  return train_loader, val_loader

def test_loader():
  transformer = A.Compose([
            #A.Resize(height=512, width=512),
            #A.RandomCrop(width=500, height=500),
            #A.Rotate(limit=15, p=.9),
            #A.VerticalFlip(p=0.1),
            A.Blur(blur_limit=1, p=0.5),
            ToTensorV2()])
  train_loader, val_loader = get_loaders(train_images[:10], train_masks[:10], val_images[:10], val_masks[:10], 1, 5, transformer, transformer, 2)
  print(train_loader, val_loader)

if __name__ == '__main__':
  pass
  #test_loader()

In [ ]:
# Checking accuracy 
def check_accuracy(model, loader, device="cuda", SMOOTH = 1e-8):
  torch.cuda.empty_cache()
  print('[*] Evaluating the model')

  # Met
  pixAcc_score = 0
  dice_score = 0
  iou_score = 0
  jaccard_score = 0

  pres_score = 0
  sens_score = 0
  spec_score = 0
  g_mean_score = 0

  model.eval()

  with torch.no_grad():
    for x, y in loader:
      if torch.cuda.is_available():
        x = x.cuda().float()
      else:
        x = x.float()

      y = y.squeeze(1).int().to('cuda')
      preds = model(x)
      x.detach_()

      preds = torch.sigmoid(preds).squeeze(1)
      preds = (preds >= 0.5).int()

      cn = confusion_matrix(torch.flatten(y).cpu(), torch.flatten(preds).cpu())
      # Precision
      pres = cn[0][0] / (cn[0][0]+cn[0][1])
      # Sensitivity
      sens = cn[0][0] / (cn[0][0]+cn[1][1])
      # Specificity
      spec = cn[1][0] / (cn[1][0] + cn[0][1])
      # G-means
      g_mean_score += math.sqrt(sens * spec)
      # Adding the scores
      pres_score += pres
      sens_score += sens
      spec_score += spec

      # Pixel Accuracy
      num_correct = (preds == y).sum()
      num_pixels = torch.numel(preds)

      pixAcc_score += num_correct/num_pixels

      # IoU: The area of overlap between the predicted segmentation and the ground truth divided by the area of union between the predicted segmentation and the ground truth
      intersection = (preds & y).float().sum()
      union = (preds | y).float().sum()
      iou = (intersection + SMOOTH) / (union + SMOOTH)
      iou_score += iou

      # # Jaccard Index
      sum = preds.sum() + y.sum() - intersection
      jaccard_score += intersection / (sum + SMOOTH)

      # Dice Coefficient (F1 Score): 2 * The area of Overlap divided by the total number of pixels in both images
      dice_score += (2 * (preds * y).sum()) / ((preds + y).sum() + SMOOTH)
      plt.imshow(preds[0].cpu(), cmap="gray", origin="lower")
      preds.detach_()
      y.detach_()
      del y, preds
      torch.cuda.empty_cache()
    print(f'[!] Pixel Accuracy: {pixAcc_score / len(loader)}')
    print(f'[!] Dice Coef, F1: {dice_score/len(loader)}')
    print(f'[!] IoU: {iou_score / len(loader)}')
    print(f'[!] Jaccard: {jaccard_score / len(loader)}')
    print(f'[!] Precision: {pres_score / len(loader)}')
    print(f'[!] Sensitivity: {sens_score / len(loader)}')
    print(f'[!] Specificity: {spec_score / len(loader)}')
    print(f'[!] G-means: {g_mean_score / len(loader)}')
    model.train()

    return {           
        'pixAcc': pixAcc_score / len(loader),
        'dice': dice_score/len(loader),
        # 'iou': iou_score / len(loader),
        # 'jaccard': jaccard_score / len(loader)
    }

    

def test_check_acc():
  transformer = A.Compose([
            A.Resize(height=160, width=160),
            #A.RandomCrop(width=500, height=500),
            #A.Rotate(limit=15, p=.9),
            #A.VerticalFlip(p=0.1),
            #A.Blur(blur_limit=1, p=0.5),
            A.Normalize(
                mean=[0.0],
                std=[1.0],
                max_pixel_value=1650.0,
            ),
            ToTensorV2()])
  train_loader, test_loader = get_loaders(train_images[:10], train_masks[:10], val_images, val_masks, 1, 5, transformer, transformer, 2)
  model = ONET(in_channels=1, out_channels=1).to('cuda')
  load_checkpoint(torch.load(f'texas/rev_onet_norm/checkpoint_13.pth.tar'), model)
  check_accuracy(model, test_loader, 'cuda')

if __name__ == '__main__':
  pass
  #test_check_acc()

In [ ]:
# Saving pred images
def summarize_performance(step, model, loader, folder="saved_images/", device="cuda"):
  # Checking the accuracy
  eval_metrics = check_accuracy(model, loader, device=device)
  torch.cuda.empty_cache()
  model.eval()
  print('[*] Saving samples')
  with torch.no_grad():
    for x, y in loader:
      if torch.cuda.is_available():
        x = x.cuda().float()
      else:
        x = x.float()
      preds = torch.sigmoid(model(x))
      n = x.shape[0]
      preds = (preds > 0.5).float()
      dif = (y-preds.cpu()).abs()
      # Plot images
      fig, axes = plt.subplots(n, 4, figsize=(12, 25))
      
      axes[0][0].axis("off")
      axes[0][0].invert_yaxis()
      axes[0][0].text(0.5,0.5, "(a) Image", size=15, ha="center", transform=axes[0][0].transAxes,  verticalalignment="top", weight="bold")
      axes[0][1].axis("off")
      axes[0][1].invert_yaxis()
      axes[0][1].text(0.5, 0.5, "(b) Ground-Truth", size=15, ha="center", transform=axes[0][1].transAxes,  verticalalignment="top", weight="bold")
      axes[0][2].axis("off")
      axes[0][2].invert_yaxis()
      axes[0][2].text(0.5,0.5, "(c) Predicted", size=15, ha="center", transform=axes[0][2].transAxes,  verticalalignment="top", weight="bold")
      axes[0][3].axis("off")
      axes[0][3].invert_yaxis()
      axes[0][3].text(0.5,0.5, "(d) Difference", size=15, ha="center", transform=axes[0][3].transAxes,  verticalalignment="top", weight="bold")
      print(n)
      for i in range(1, n):
        # Adding the real images
        axes[i][0].imshow(x[i].cpu()[0], cmap="gray", origin="lower")
        # Adding the ground-truth masks
        axes[i][1].imshow(y[i].cpu()[0], cmap="gray", origin="lower")
        # Adding the predicted masks
        axes[i][2].imshow(preds[i].cpu()[0], cmap="gray", origin="lower")
        # Adding the difference
        axes[i][3].imshow(dif[i].cpu()[0], cmap="gray", origin="lower")
      preds.detach_()
      x.detach_()
      break
  # Adjust subplot parameters to give specified padding
  plt.tight_layout()
  # Save plots to file
  filename = f'Summary_Step_{step}.png'
  plt.savefig(f'{folder}/{filename}')
  plt.close()
  model.train()

  return None

def test_sum():
  transformer = A.Compose([
            A.Resize(height=160, width=160),
            #A.RandomCrop(width=500, height=500),
            #A.Rotate(limit=15, p=.9, border_mode=cv2.BORDER_CONSTANT, value=-1300),
            #A.VerticalFlip(p=0.1),
            #A.Blur(blur_limit=1, p=0.5),
            ToTensorV2()])
  _, loader = get_loaders(train_images[:10], train_masks[:10], val_images[:10], val_masks[:10], 1, 5, transformer, transformer, 2)
  model = ONET(1,1, [8, 16]).to('cuda')
  summarize_performance(0, model, loader, folder="onet/", device='cuda')

if __name__ == '__main__':
  pass
  # test_sum()

In [ ]:
covid_images = np.load('covid_images_0.npy')
covid_masks = np.load('covid_masks_0.npy')
covid_images.shape

(1000, 512, 512)

In [ ]:
train_images = covid_images[:1100]
val_images = covid_images[1100:]

train_masks = covid_masks[:1100]
val_masks = covid_masks[1100:]
del covid_images, covid_masks
val_masks.shape, train_masks.shape

((0, 512, 512), (1000, 512, 512))

In [ ]:
# One training epoch
def train_fn(epoch, model, train_loader, val_loader, optimizer, loss_fn, scaler, writer, device, check_acc = False, folder="testing_save/"):

  loop = tqdm(train_loader)

  for batch_idx, (data, targets) in enumerate(loop):

    data = data.float().to(device=device)
    targets = targets.float().to(device=device)

    # forward
    with torch.cuda.amp.autocast():
        predictions = model(data)
        loss = loss_fn(predictions, targets)

    # backward
    optimizer.zero_grad()
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()

    # Logging
    step = (epoch-1)*len(train_loader) + batch_idx
    writer.add_scalar('Loss/train', loss, step)
    # acc = getPixAcc(train_loader, model, device=DEVICE, iterations=1)
    # writer.add_scalar('PixACC/train', acc, epoch)
    # acc = getPixAcc(val_loader, model, device=DEVICE, iterations=1)
    # writer.add_scalar('PixACC/val', acc, epoch)

    # update tqdm loop
    loop.set_postfix(loss=loss.item())

    # Cleaning
    loss.detach_()
    data.detach_()
    targets.detach_()
    predictions.detach_()
    del loss, data, targets, predictions
    torch.cuda.empty_cache()

  if check_acc:
    summarize_performance(epoch, model, val_loader, folder=folder, device=device)

In [ ]:
def train(DEVICE, LEARNING_RATE, NUM_EPOCHS, TRAIN_IMG, TRAIN_MASK, VAL_IMG, VAL_MASK, TRAIN_BATCH_SIZE, VAL_BATCH_SIZE, IMAGE_RES, CROP_SIZE, NUM_WORKERS, PIN_MEMORY, LOAD_MODEL='', FOLDER='testing_save/'):
    train_transform = A.Compose([
            A.Resize(height=IMAGE_RES, width=IMAGE_RES),
            A.RandomCrop(width=CROP_SIZE, height=CROP_SIZE),
            A.Rotate(limit=15, p=.9),
            A.VerticalFlip(p=0.1),
            #A.Blur(blur_limit=1, p=0.5),
            ToTensorV2()])

    val_transforms = A.Compose([
            A.Resize(height=IMAGE_RES, width=IMAGE_RES),
            #A.RandomCrop(width=CROP_SIZE, height=CROP_SIZE),
            #A.Rotate(limit=15, p=.9),
            #A.VerticalFlip(p=0.1),
            #A.Blur(blur_limit=1, p=0.5),
            ToTensorV2()])

    model = ONET(in_channels=1, out_channels=1).to(DEVICE)
    if LOAD_MODEL != '':
      load_checkpoint(torch.load(LOAD_MODEL), model)

    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    train_loader, val_loader = get_loaders(TRAIN_IMG, TRAIN_MASK, VAL_IMG, VAL_MASK, TRAIN_BATCH_SIZE, VAL_BATCH_SIZE, train_transform, val_transforms, NUM_WORKERS, PIN_MEMORY)

    scaler = torch.cuda.amp.GradScaler()

    WRITER = SummaryWriter(log_dir=f'tb_log/eval_master')

    for epoch in range(NUM_EPOCHS):
      print('#'*50)
      print(f'[!] Epoch: {epoch}')
      check = (epoch+1)%2 == 0
      train_fn(epoch+1, model, train_loader, val_loader, optimizer, loss_fn, scaler, writer=WRITER, device=DEVICE, check_acc=True, folder=FOLDER)

In [ ]:
DEVICE = "cuda" #@param ['cpu', 'cuda']
LEARNING_RATE = 0.0001 #@param [1e-2, 1e-3, 1e-4, 1e-5] {type: 'raw'}
NUM_EPOCHS =   20 #@param {type: 'number'}
TRAIN_IMG = train_images
TRAIN_MASK = train_masks
VAL_IMG = val_images
VAL_MASK = val_masks
TRAIN_BATCH_SIZE = 4 #@param {type:"slider", min:1, max:100, step:1}
val_BATCH_SIZE = 8 #@param {type:"slider", min:1, max:100, step:1}
IMAGE_RES =  256 #@param {type: 'number'}
CROP_SIZE =  256 #@param {type: 'number'}
NUM_WORKERS = 2 #@param {type:"slider", min:1, max:8, step:1}
PIN_MEMORY = True #@param {type:"boolean"}
LOAD_MODEL = '' #@param {type: 'string'}
FOLDER = 'eval_master/' #@param {type: 'string'}

train(
    DEVICE,
    LEARNING_RATE,
    NUM_EPOCHS,
    TRAIN_IMG,
    TRAIN_MASK,
    VAL_IMG,
    VAL_MASK,
    TRAIN_BATCH_SIZE,
    val_BATCH_SIZE,
    IMAGE_RES,
    CROP_SIZE,
    NUM_WORKERS,
    PIN_MEMORY,
    LOAD_MODEL,
    FOLDER
)